In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [2]:
sys.path = ['/home/etsymbalov/.local/lib/python3.6/site-packages'] + sys.path

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from experiment_setup import get_model, set_random, build_estimator
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from dataloader.toy import ToyQubicData, ToySinData
from model.mlp import MLP
from model.ensemble import MLPEnsemble
import pickle

plt.rcParams['figure.facecolor'] = 'white'

In [4]:
from dataloader.builder import build_dataset
from model.mlp import MLP
from uncertainty_estimator.masks import build_masks 
from experiment_setup import build_estimator
from analysis.metrics import get_uq_metrics
from experiments.utils.data import scale, split_ood, multiple_kfold

In [8]:
import os
import pickle
import pandas as pd

In [15]:
def construct_estimator(model, model_type, name):
    if model_type == 'mask': 
        mask = masks[name]
        return build_estimator(
            'mcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])
    elif model_type == 'emask': 
        mask = emasks[name]
        return build_estimator(
            'emcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])
    else:
        return build_estimator(name, model)


class Evaluator:    
    def __init__(self, x_test, y_test, y_scaler, tag='standard'):
        self.x_test = x_test
        self.y_test = y_test
        self.unscale = lambda y : y_scaler.inverse_transform(y) 
        self.tag = tag
        self.results = []

    def bench(self, model, name, model_type='mask'): 
        predictions = model(self.x_test).cpu().numpy()
        
        errors = np.abs(predictions - self.y_test)
        
        scaled_errors = self.unscale(predictions) - self.unscale(self.y_test)
        rmse = np.sqrt(np.mean(np.square(scaled_errors)))

        estimator = construct_estimator(model, model_type, name)
        if model_type == 'emask':
            name = 'e_' + name
        
        for run in range(config['n_ue_runs']):
            estimations = estimator.estimate(self.x_test)
            acc, ndcg, ll = get_uq_metrics(estimations, errors, 
                                           config['acc_percentile'],
                                           bins = [80, 95, 99]
                                          )
            self.results.append([acc, ndcg, ll, rmse, name, self.tag])
            if hasattr(estimator, 'reset'):
                estimator.reset()

In [16]:
folder = './exp1/'
files = os.listdir(folder)

In [ ]:
data = []
for cnt, file in enumerate(files):
    with open('log_exp.log', 'w') as f:
        f.write(f'{cnt} / {len(files)}')
    with open(folder + file, 'rb') as f:
        dct = pickle.load(f)
#     print(file)
    config = dct['config']
    config['n_ue_runs'] = 1
    config['acc_percentile'] = .1
    state_dict = dct['state_dict']
    x_train, y_train, x_val, y_val, x_scaler, y_scaler = dct['data']
    
    model = MLPEnsemble(config['layers'], 
                    n_models=config['n_ens'], 
                    reduction='mean')
    model.load_state_dict(state_dict)
    
    standard_evaluator = Evaluator(x_val, y_val, y_scaler, 'standard')
    masks = build_masks(DEFAULT_MASKS)
    emasks = [build_masks(DEFAULT_MASKS) for i in range(config['n_ens'])]
    emasks = {key: [e[key] for e in emasks] for key in masks.keys()}

    single_model = model.models[2]
    for name in masks: 
#         print(name, end = '|')
        standard_evaluator.bench(single_model, name, 'mask')
    standard_evaluator.bench(model, 'eue', 'ensemble')    
    for name in emasks: 
#         print(name, end = '*|')
        standard_evaluator.bench(model, name, 'emask')
    mask_df = pd.DataFrame(standard_evaluator.results, 
                       columns=['Acc', 'NDCG', 'LL', 'RMSE', 'Mask', 'Tag'])
    mask_df['fname'] = file
    data.append(mask_df)
    pd.concat(data).to_csv('exper1_results.csv', index = None)